<a href="https://colab.research.google.com/github/saulosma/ML-e-Psicologia/blob/main/Auto_Arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------- AUTO ARIMA INTELIGENTE --------------------------- #
# Script completo que:
# - Testa a estacionariedade da série temporal
# - Aplica diferenciação automática, se necessário
# - Executa a busca automática pelo melhor modelo ARIMA/SARIMA
# ---------------------------------------------------------------------------- #

import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.stattools import adfuller

# ========================== CONFIGURAÇÕES INICIAIS =========================== #

# >>>> ALTERE APENAS ESTA SEÇÃO  <<<<
base = base_nucleos  # Substitua pelo nome da sua série temporal (pandas.Series)

janela = 24   # tamanho da janela de treino (ex.: 24 meses)
h = 12         # horizonte de previsão (ex.: 12 meses à frente)
s = 12        # sazonalidade (12 = mensal)
#-> ADAPTE A SUA NECESSIDADE DE PREVISÃO
# Espaço de busca (p, d, q, P, D, Q)
p = d = q = range(0, 3)
P = D = Q = range(0, 2)

# ========================= FUNÇÃO DE ESTAIONARIEDADE ========================= #

def testar_estacionariedade(serie, nome="Série"):
    """
    Testa a estacionariedade da série usando o teste ADF (Dickey-Fuller Aumentado).
    Retorna True se a série for estacionária, False caso contrário.
    """
    serie = serie.dropna()
    resultado = adfuller(serie)
    estatistica, p_valor = resultado[0], resultado[1]

    print(f"\n🔎 Teste ADF para {nome}:")
    print(f"  Estatística ADF: {estatistica:.4f}")
    print(f"  Valor-p: {p_valor:.4f}")

    if p_valor < 0.05:
        print("✅ A série é estacionária (nível de significância de 5%)")
        return True
    else:
        print("⚠️ A série NÃO é estacionária. Aplicando diferenciação...")
        return False

# ====================== AJUSTE AUTOMÁTICO DE ESTAIONARIEDADE ====================== #

base_diff = base.copy()

# Teste da série original
if not testar_estacionariedade(base_diff, "Série Original"):
    base_diff = base_diff.diff().dropna()
    if not testar_estacionariedade(base_diff, "1ª Diferenciação"):
        # Caso ainda não seja estacionária, aplica diferenciação sazonal
        base_diff = base_diff.diff(s).dropna()
        testar_estacionariedade(base_diff, "Diferenciação Sazonal")

print("\n📊 Série pronta para modelagem ARIMA/SARIMA!")

# =========================== PREPARAÇÃO DO LOOP ============================== #

pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q))

resultados = []

# =============================== LOOP AUTO ARIMA ============================== #

for ordem in pdq:
    for ordem_sazonal in seasonal_pdq:
        previsoes = []
        reais = []

        for i in range(janela, len(base_diff) - h + 1):
            treino = base_diff[i-janela:i]
            teste = base_diff[i:i+h]

            try:
                modelo = SARIMAX(
                    treino,
                    order=ordem,
                    seasonal_order=(ordem_sazonal[0],
                                    ordem_sazonal[1],
                                    ordem_sazonal[2],
                                    s),
                    enforce_stationarity=False,
                    enforce_invertibility=False
                )
                resultado = modelo.fit(disp=False)
                forecast = resultado.forecast(steps=h)

                valor_previsto = float(forecast.values.ravel()[-1])
                valor_real = float(teste.values.ravel()[-1])

                if not np.isnan(valor_previsto) and not np.isnan(valor_real):
                    previsoes.append(valor_previsto)
                    reais.append(valor_real)

            except Exception:
                continue

        # Avaliação das previsões
        if len(previsoes) > 0:
            previsoes = np.array(previsoes, dtype=float)
            reais = np.array(reais, dtype=float)

            mask = ~np.isnan(previsoes) & ~np.isnan(reais)
            previsoes = previsoes[mask]
            reais = reais[mask]

            if len(previsoes) > 0:
                mae = mean_absolute_error(reais, previsoes)
                rmse = mean_squared_error(reais, previsoes) ** 0.5

                resultados.append({
                    "ordem": ordem,
                    "ordem_sazonal": ordem_sazonal,
                    "MAE": mae,
                    "RMSE": rmse
                })

# ============================= RESULTADOS FINAIS ============================= #

if len(resultados) == 0:
    print("⚠️ Nenhum modelo pôde ser ajustado. Verifique a base ou os parâmetros.")
else:
    resultados_ordenados = sorted(resultados, key=lambda x: x["MAE"])

    print("\n🏆 Top 10 modelos (menor MAE):")
    for r in resultados_ordenados[:10]:
        print(f"ARIMA{r['ordem']} x SARIMA{r['ordem_sazonal']}12 "
              f"-> MAE: {r['MAE']:.4f}, RMSE: {r['RMSE']:.4f}")

print("\n✅ Processo concluído com sucesso!")

# ---------------------------------------------------------------------------- #
# NOTAS:
# - Teste de estacionariedade automático (ADF)
# - Aplicação automática de diferenciação se necessário
# - Feedback visual e estruturado sobre a preparação da série
# - Altere apenas o nome da variável 'base' no início do script.
# - Certifique-se de que 'base' é uma série temporal univariada (pandas.Series).
# - 'janela' define o número de observações para treino.
# - 'h' define o horizonte de previsão.
# - Ajuste p, d, q, P, D, Q e s conforme o padrão dos seus dados, se necessario.
# - Quanto maior o espaço de busca, mais lento será o processamento.
# ---------------------------------------------------------------------------- #